<a href="https://colab.research.google.com/github/NikitaKholod/Tutorials_materials/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Tutorial_1_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Отчет по выполнению Туториала 1.

Выполните код ниже для установки необходимых пакетов

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

ModuleNotFoundError: No module named 'distutils'

In [ ]:
!curl -L -o tutorial_packages.yml "https://drive.google.com/uc?export=download&id=1gfDrV8Iiv5C3OrsDyuVzmluRGs2PwC92"

In [ ]:
!conda env update -n base --file tutorial_packages.yml

Ниже вводите код, необходимый для выполнения заданий

In [ ]:
!curl -L -o druglike_small_molecules.csv "https://drive.google.com/uc?export=download&id=1ZYIBj3EwK4uctWZCFO7seLdkn6R5K75R"

## Задание 1

### Пункт 1

In [ ]:
import pandas as pd

In [ ]:
#Переводим библиотеку из .csv в формат DataFrame:
molecules = pd.read_csv('druglike_small_molecules.csv', index_col='ID')
#Смотрим, что получилось:
molecules.head()

,FDA_APPROVED,SMILES
ID,,
1,No,CC(C)CN1c2c(c(n(n2)Cc3cccc4c3cccc4)c5ccncc5)C(...
2,No,c1ccc(cc1)[C@@H](C(=O)O)N
3,No,Cc1cccc(c1OCC(=O)N[C@@H](Cc2ccccc2)[C@@H](C(=O...
4,No,c1ccc(cc1)C2(CCCC2)CN
5,No,c1ccc2c(c1)C[C@H](N(C2)C(=O)C[C@@H](Cc3ccccc3F...


### Пункт 2

In [ ]:
from rdkit.Chem import PandasTools

In [ ]:
# Добавляем к molecules колонку с объектами типа 'RDKit molecule':
PandasTools.AddMoleculeColumnToFrame(molecules,
                                     'SMILES',
                                     'ROMol',
                                     includeFingerprints=True)

[07:56:09] Explicit valence for atom # 5 C, 6, is greater than permitted
[07:56:16] Explicit valence for atom # 7 Cl, 5, is greater than permitted


In [ ]:
# Проверяем, что все прошло нормально:
molecules.head()

,FDA_APPROVED,SMILES,ROMol
ID,,,
1,No,CC(C)CN1c2c(c(n(n2)Cc3cccc4c3cccc4)c5ccncc5)C(...,<rdkit.Chem.rdchem.Mol object at 0x7db69b31a880>
2,No,c1ccc(cc1)[C@@H](C(=O)O)N,<rdkit.Chem.rdchem.Mol object at 0x7db690eb25e0>
3,No,Cc1cccc(c1OCC(=O)N[C@@H](Cc2ccccc2)[C@@H](C(=O...,<rdkit.Chem.rdchem.Mol object at 0x7db690eb2500>
4,No,c1ccc(cc1)C2(CCCC2)CN,<rdkit.Chem.rdchem.Mol object at 0x7db690eb2e30>
5,No,c1ccc2c(c1)C[C@H](N(C2)C(=O)C[C@@H](Cc3ccccc3F...,<rdkit.Chem.rdchem.Mol object at 0x7db690eb2420>


### Пункт 3

In [ ]:
# Фильтруем строки с None, находим их индексы и удаляем их из molecules:
to_del_idx = molecules[molecules['ROMol'].isna()].index
molecules = molecules.drop(index=to_del_idx)

### Пункт 4

In [ ]:
# Удаляем текущий индекс:
molecules = molecules.reset_index().drop(columns=['ID'])

In [ ]:
# Создаем колонку для будущего индекса:
molecules['ID'] = range(1, molecules.shape[0]+1)

In [ ]:
# Устанавливаем индекс:
molecules = molecules.set_index('ID')

In [ ]:
# Проверяем, что все нормально:
molecules.tail()

,FDA_APPROVED,SMILES,ROMol
ID,,,
10963,No,CC(C)n1c2c(c(n1)C#Cc3cccnc3)c(ncn2)N,<rdkit.Chem.rdchem.Mol object at 0x7db5a3ad5cb0>
10964,No,Cc1ccccc1N2C(=Nc3cccc(c3C2=O)C)Cn4c5c(c(n4)c6c...,<rdkit.Chem.rdchem.Mol object at 0x7db5a3ad5d90>
10965,No,Cc1ccccc1N2C(=Nc3cccc(c3C2=O)C)Cn4c5c(c(n4)c6c...,<rdkit.Chem.rdchem.Mol object at 0x7db5a3ad5e70>
10966,No,COc1ccccc1N2C(=O)C3=C(CCCC3)N=C2SCC(=O)Nc4ccc5...,<rdkit.Chem.rdchem.Mol object at 0x7db5a3ad5f50>
10967,No,c1ccc(c(c1)[N+](=O)[O-])S(=O)(=O)n2ccc3c2cc(cn...,<rdkit.Chem.rdchem.Mol object at 0x7db5a3ad6030>


## Задание*

### Пункт 1

In [ ]:
from rdkit.Chem import (AddHs, AllChem, SDWriter, SDMolSupplier)

In [61]:
# @title Считать ли результат применения метода EmbedMolecule()?

Compute = True # @param {"type":"boolean"}
if Compute:

  # Создаю объект Series молекул с добавленными H:
  mol_with_hs = molecules['ROMol'].apply(AddHs)

  # Создаю массив result с результатом расчетов и 3D молекулы в DataFrame:
  result = []
  for i in range(1, mol_with_hs.shape[0]+1):
    mol = mol_with_hs[i]
    result.append(AllChem.EmbedMolecule(mol, randomSeed=42))
    molecules.at[i, 'ROMol'] = mol

else:
  print('Отдыхаем!')

[08:07:37] Interrupted, cancelling conformer generation


KeyboardInterrupt: Embedding cancelled

Чтобы не считать каждый раз при перезапуске сеанса 10 минут результат, я сохранил его в качестве текстого файла и загрузил на GitHub.

```python
with open('result.txt', 'w') as file:
    file.writelines(str(result))

writer = SDWriter('molecules_3d.sdf')
for mol in molecules['ROMol']:
    writer.write(mol)
writer.close()
```


In [64]:
# В случае, если не хочется заново считать результат, пользуемся готовым:
try:
  res = pd.Series(result, index=range(1, len(result)+1))
  molecules = molecules[res == 0]
except NameError:
  import requests

  url = 'https://raw.githubusercontent.com/NikitaKholod/Tutorials_materials/refs/heads/main/result.txt'
  result_text = requests.get(url).text
  result_copy = [int(x) for x in result_text[1:-1].split(', ')]
  res = pd.Series(result_copy, index=range(1, len(result_copy)+1))

  url = 'https://github.com/NikitaKholod/Tutorials_materials/releases/download/%23Data/molecules_3d.sdf'
  molecules_3d = requests.get(url).text
  supplier = SDMolSupplier()
  supplier.SetData(molecules_3d)
  mols = [mol for mol in supplier if mol is not None]
  molecules['ROMol'] = mols
  molecules = molecules[res == 0]

### Пункт 2

In [65]:
molecules = molecules.reset_index().drop(columns=['ID'])
molecules['ID'] = range(1, molecules.shape[0]+1)
molecules = molecules.set_index(['ID'])

In [73]:
# @title Захотелось посмотреть на 3D-модели молекул, чтобы убедиться, чтобы все хорошо:
Index_of_mol = 10949 # @param {"type":"number","placeholder":"Введите целое число от 1 до 10948"}
if Index_of_mol in range(1, 10949):
  try:
    import py3Dmol
  except ImportError:
    !pip install py3Dmol
    import py3Dmol

  mol = molecules.iloc[Index_of_mol-1]['ROMol']
  mol_block = AllChem.MolToMolBlock(mol)

  viewer = py3Dmol.view(width=400, height=300)
  viewer.addModel(mol_block, 'mol')
  viewer.setStyle({'stick': {}})
  viewer.zoomTo()
  viewer.show()
else:
  print('Введено неправильное число!')

Введено неправильное число!


## Задание 2

10948